**  Câu 1: Hãy viết câu lệnh SQL để tính sự tương quan giữa A và B theo công thức sau:


In [5]:
import sqlite3
import pandas as pd
import math
import scipy.stats as stats

# Tạo kết nối SQLite trong bộ nhớ
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Câu 1: Tính hệ số tương quan giữa A và B bằng SQL
cursor.execute('''
    CREATE TABLE Scores (
        day TEXT,
        A REAL,
        B REAL
    )
''')

data = [
    ("Day 1", 8, 9),
    ("Day 2", 7.5, 8.5),
    ("Day 3", 6, 7),
    ("Day 4", 7, 6)
]
cursor.executemany("INSERT INTO Scores VALUES (?, ?, ?)", data)
conn.commit()

conn.create_function("SQRT", 1, math.sqrt)

query_corr = '''
SELECT
    (COUNT(*) * SUM(A * B) - SUM(A) * SUM(B)) /
    (SQRT(COUNT(*) * SUM(A * A) - SUM(A) * SUM(A)) *
     SQRT(COUNT(*) * SUM(B * B) - SUM(B) * SUM(B))) AS correlation_AB
FROM Scores
'''
result_corr = cursor.execute(query_corr).fetchone()
print("Correlation A vs B:", result_corr[0])

Correlation A vs B: 0.6910506641398757


** Câu 2: Một công ty oto đang kiểm tra 3 loại mẫu mới A, B và C trong 4 ngày, và chấm điểm theo thang từ 1 đến 10 điểm cho mỗi ngày với bảng sau. Liệu có sự khác biệt đáng kể giữa các mẫu dựa trên điểm số mà chúng nhận được trong 4 ngày thử nghiệm không? Kết quả thử nghiệm phụ thuộc vào ngày hay phụ thuộc vào mẫu xe? Hãy chuyển đổi dữ liệu sang dạng quan hệ và thực hiện kiểm tra χ2.

In [9]:
import sqlite3
import pandas as pd
import scipy.stats as stats

# Kết nối SQLite 
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Tạo bảng dữ liệu
cursor.execute("""
    CREATE TABLE car_tests (
        Day TEXT,
        Model TEXT,
        Score REAL
    )
""")

# Dữ liệu thử nghiệm
data = [
    ("Day 1", "A", 8), ("Day 1", "B", 9), ("Day 1", "C", 7),
    ("Day 2", "A", 7.5), ("Day 2", "B", 8.5), ("Day 2", "C", 7),
    ("Day 3", "A", 6), ("Day 3", "B", 7), ("Day 3", "C", 8),
    ("Day 4", "A", 7), ("Day 4", "B", 6), ("Day 4", "C", 5)
]

# Chèn dữ liệu vào bảng
cursor.executemany("INSERT INTO car_tests VALUES (?, ?, ?)", data)
conn.commit()
# Truy vấn dữ liệu từ SQLite
query = "SELECT Day, Model, Score FROM car_tests"
df = pd.read_sql_query(query, conn)

# Chuyển dữ liệu thành bảng tần suất 
table = pd.pivot_table(df, values="Score", index="Day", columns="Model", aggfunc="sum")
# Kiểm định Chi-square
chi2_stat, p_value, dof, expected = stats.chi2_contingency(table)

# In kết quả
print(f"Chi-square Statistic: {chi2_stat}")
print(f"P-value: {p_value}")
print(f"Degrees of Freedom: {dof}")
print("Expected Frequencies:")
print(pd.DataFrame(expected, index=table.index, columns=table.columns))
# Kiểm tra kết quả
alpha = 0.05
if p_value < alpha:
    print("Có sự khác biệt đáng kể giữa các mẫu xe.")
else:
    print("Không có đủ bằng chứng để khẳng định sự khác biệt giữa các mẫu xe.")

# Đóng kết nối SQLite
conn.close()

Chi-square Statistic: 0.826439691310499
P-value: 0.9913459315666546
Degrees of Freedom: 6
Expected Frequencies:
Model         A         B         C
Day                                
Day 1  7.953488  8.511628  7.534884
Day 2  7.622093  8.156977  7.220930
Day 3  6.959302  7.447674  6.593023
Day 4  5.965116  6.383721  5.651163
Không có đủ bằng chứng để khẳng định sự khác biệt giữa các mẫu xe.


** Câu 3: Bảng flights(departure_time,...) chứa các giá trị thời gian dưới dạng số nguyên ví dụ 830 cho 8:30AM, 1445 cho 2:45PM).Hãy chuyển dổi các giá trị này thành định dạng thời gian.

In [23]:
import sqlite3
import random

# Hàm tạo thời gian ngẫu nhiên (ví dụ: 1345)
def generate_time():
    hour = random.randint(0, 23)
    minute = random.randint(0, 59)
    return hour * 100 + minute

# Chuyển đổi thời gian từ int sang định dạng chuỗi HH:MM
def convert_to_time(time_int):
    hour = time_int // 100
    minute = time_int % 100
    return f"{hour:02d}:{minute:02d}"

# Dùng with để đảm bảo kết nối được đóng tự động
with sqlite3.connect("flights.db", timeout=5) as conn:
    cursor = conn.cursor()

    # Tạo bảng nếu chưa tồn tại
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS flights (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        departure_time_int INTEGER,
        departure_time_str TEXT
    )
    """)

    # Xóa toàn bộ dữ liệu cũ để tránh trùng lặp
    cursor.execute("DELETE FROM flights")

    # Tạo 50 chuyến bay với thời gian ngẫu nhiên
    departure_times = []
    for _ in range(10):
        time_int = generate_time()
        time_str = convert_to_time(time_int)
        departure_times.append((time_int, time_str))

    # Chèn dữ liệu vào bảng
    cursor.executemany(
        "INSERT INTO flights (departure_time_int, departure_time_str) VALUES (?, ?)",
        departure_times
    )
    conn.commit()

    # Truy vấn lại để hiển thị dữ liệu
    cursor.execute("SELECT id, departure_time_int, departure_time_str FROM flights LIMIT 50")
    rows = cursor.fetchall()

# Hiển thị kết quả
print("ID | Departure Time (Int) | Departure Time (Str)")
print("-" * 40)
for row in rows:
    print(f"{row[0]:<3} | {row[1]:<18} | {row[2]}")


ID | Departure Time (Int) | Departure Time (Str)
----------------------------------------
51  | 257                | 02:57
52  | 233                | 02:33
53  | 2231               | 22:31
54  | 511                | 05:11
55  | 2144               | 21:44
56  | 1105               | 11:05
57  | 1007               | 10:07
58  | 323                | 03:23
59  | 1830               | 18:30
60  | 723                | 07:23


** Câu 5 : hãy xác định liệu hai người trong bảng Patient(last_name, weight, height) có phải là một người hay không bằng cách sử dụng khảng cách kết hợp BOOLEAN trên "last_name" và " weight"

In [17]:
import sqlite3
import pandas as pd
import math
import scipy.stats as stats

# Tạo kết nối SQLite trong bộ nhớ
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Câu 1: Tính hệ số tương quan giữa A và B bằng SQL
cursor.execute('''
    CREATE TABLE Scores (
        day TEXT,
        A REAL,
        B REAL
    )
''')

data = [
    ("Day 1", 8, 9),
    ("Day 2", 7.5, 8.5),
    ("Day 3", 6, 7),
    ("Day 4", 7, 6)
]
cursor.executemany("INSERT INTO Scores VALUES (?, ?, ?)", data)
conn.commit()

conn.create_function("SQRT", 1, math.sqrt)

query_corr = '''
SELECT
    (COUNT(*) * SUM(A * B) - SUM(A) * SUM(B)) /
    (SQRT(COUNT(*) * SUM(A * A) - SUM(A) * SUM(A)) *
     SQRT(COUNT(*) * SUM(B * B) - SUM(B) * SUM(B))) AS correlation_AB
FROM Scores
'''
result_corr = cursor.execute(query_corr).fetchone()
print("Correlation A vs B:", result_corr[0])


# Câu 2: Chuyển dữ liệu sang dạng quan hệ & kiểm định χ²
cursor.execute('''
    CREATE TABLE Scores_ABC (
        day TEXT,
        A REAL,
        B REAL,
        C REAL
    )
''')

data_abc = [
    ("Day 1", 8, 9, 7),
    ("Day 2", 7.5, 8.5, 7),
    ("Day 3", 6, 7, 8),
    ("Day 4", 7, 6, 5)
]
cursor.executemany("INSERT INTO Scores_ABC VALUES (?, ?, ?, ?)", data_abc)
conn.commit()

df_abc = pd.read_sql_query("SELECT * FROM Scores_ABC", conn)
df_long = df_abc.melt(id_vars=["day"], var_name="model", value_name="score")
df_long["score_cat"] = pd.cut(df_long["score"], bins=[0, 6, 8, 10], labels=["Low", "Medium", "High"])

contingency = pd.crosstab(df_long["model"], df_long["score_cat"])
chi2, p, _, _ = stats.chi2_contingency(contingency)
print("Chi-square:", chi2, "| p-value:", p)


# Câu 3: Chuyển giá trị thời gian thành định dạng chuẩn
cursor.execute("CREATE TABLE Flights (departure_time INTEGER)")
cursor.executemany("INSERT INTO Flights (departure_time) VALUES (?)", [(830,), (1445,), (30,), (0,)])
conn.commit()

query_time = '''
SELECT departure_time,
       printf('%02d:%02d', departure_time / 100, departure_time % 100) AS time_formatted
FROM Flights
'''
df_time = pd.read_sql_query(query_time, conn)
print(df_time)


# Câu 4: Tìm giá trị ngoại lệ bằng MAD
data_mad = [10, 12, 13, 10, 100, 11, 12]
df_mad = pd.DataFrame({"value": data_mad})
median = df_mad["value"].median()
mad = (df_mad["value"] - median).abs().median()
threshold = 1.5 * mad
df_mad["is_outlier"] = (df_mad["value"] - median).abs() > threshold
print(df_mad)


# Câu 5: Kiểm tra trùng lặp theo last_name và weight
cursor.execute('''
    CREATE TABLE Patient (
        last_name TEXT,
        weight INTEGER,
        height INTEGER
    )
''')

data_patient = [
    ("Nguyen", 70, 170),
    ("Tran", 60, 165),
    ("Nguyen", 70, 171),
    ("Le", 80, 180),
    ("Tran", 65, 167)
]
cursor.executemany("INSERT INTO Patient VALUES (?, ?, ?)", data_patient)
conn.commit()

query_dup = '''
SELECT *
FROM Patient
WHERE (last_name, weight) IN (
    SELECT last_name, weight
    FROM Patient
    GROUP BY last_name, weight
    HAVING COUNT(*) > 1
)
'''
df_duplicates = pd.read_sql_query(query_dup, conn)
print(df_duplicates)


Correlation A vs B: 0.6910506641398757
Chi-square: 5.142857142857143 | p-value: 0.2729510249670288
   departure_time time_formatted
0             830          08:30
1            1445          14:45
2              30          00:30
3               0          00:00
   value  is_outlier
0     10        True
1     12       False
2     13       False
3     10        True
4    100        True
5     11       False
6     12       False
  last_name  weight  height
0    Nguyen      70     170
1    Nguyen      70     171


In [ ]:
** Câu 4: Viết truy vấn SQL để tìm các ngoại lệ bằng cách sử dụng MAD. Một quy tắc chung là xem xét các giá trị ngoại lẹ lớn hơn 1.5 lần so với giá trị MAD, trong đó x là số độ lệch chuẩn mà ta coi là có ý nghĩa.

In [21]:
import sqlite3
import random
import pandas as pd

# Kết nối tới cơ sở dữ liệu SQLite 
conn = sqlite3.connect(':memory:')  
cursor = conn.cursor()

# Tạo bảng
cursor.execute('''
CREATE TABLE data (
    id INTEGER PRIMARY KEY,
    value REAL
);
''')

# Chèn dữ liệu ngẫu nhiên vào bảng
for i in range(10):
    value = round(random.uniform(0, 10), 2)  
    cursor.execute('INSERT INTO data (id, value) VALUES (?, ?)', (i + 1, value))

# Tính giá trị trung vị (Median) sử dụng ROW_NUMBER()
cursor.execute('''
WITH RankedData AS (
    SELECT value,
           ROW_NUMBER() OVER (ORDER BY value) AS row_num,
           COUNT(*) OVER () AS total_count
    FROM data
)
SELECT value
FROM RankedData
WHERE row_num = (total_count + 1) / 2;
''')
median_value = cursor.fetchone()[0]

# Tính độ lệch tuyệt đối 
cursor.execute('''
WITH RankedData AS (
    SELECT value,
           ROW_NUMBER() OVER (ORDER BY value) AS row_num,
           COUNT(*) OVER () AS total_count
    FROM data
),
deviation AS (
    SELECT value, ABS(value - (SELECT value FROM RankedData WHERE row_num = (total_count + 1) / 2)) AS abs_deviation
    FROM RankedData
)
SELECT abs_deviation FROM deviation;
''')

# Lấy tất cả các độ lệch tuyệt đối và tính MAD
deviations = [row[0] for row in cursor.fetchall()]
mad_value = sorted(deviations)[len(deviations) // 2]  # MAD là trung vị của độ lệch tuyệt đối

# In giá trị MAD
print(f"Giá trị MAD: {mad_value}")

# Tạo danh sách ngoại lệ và không phải ngoại lệ
cursor.execute('''
WITH RankedData AS (
    SELECT value,
           ROW_NUMBER() OVER (ORDER BY value) AS row_num,
           COUNT(*) OVER () AS total_count
    FROM data
),
deviation AS (
    SELECT value, ABS(value - (SELECT value FROM RankedData WHERE row_num = (total_count + 1) / 2)) AS abs_deviation
    FROM RankedData
)
SELECT id, value, ABS(value - (SELECT value FROM RankedData WHERE row_num = (total_count + 1) / 2)) AS abs_deviation
FROM data;
''')

# Lấy kết quả
rows = cursor.fetchall()

# Phân loại ngoại lệ và không phải ngoại lệ
outliers = []
non_outliers = []

for row in rows:
    id, value, abs_deviation = row
    if abs_deviation > 1.5 * mad_value:
        outliers.append((id, value))
    else:
        non_outliers.append((id, value))

# Tạo DataFrame từ các danh sách ngoại lệ và không phải ngoại lệ
df_outliers = pd.DataFrame(outliers, columns=['id', 'Ngoại lệ'])
df_non_outliers = pd.DataFrame(non_outliers, columns=['id', 'Không phải ngoại lệ'])
df_combined = pd.merge(df_outliers, df_non_outliers, left_index=True, right_index=True, how='outer')

# in toàn bộ DataFrame
pd.set_option('display.max_rows', None)  # Hiển thị tất cả các dòng
pd.set_option('display.max_columns', None)  # Hiển thị tất cả các cột
pd.set_option('display.width', None)  # Không giới hạn chiều rộng hiển thị
print("\nDataFrame chứa giá trị ngoại lệ và không phải ngoại lệ:")
print(df_combined)

# Đóng kết nối
conn.close()

Giá trị MAD: 2.1500000000000004

DataFrame chứa giá trị ngoại lệ và không phải ngoại lệ:
   id_x  Ngoại lệ  id_y  Không phải ngoại lệ
0   8.0      3.17     1                 7.99
1   NaN       NaN     2                 5.27
2   NaN       NaN     3                 8.58
3   NaN       NaN     4                 6.81
4   NaN       NaN     5                 8.82
5   NaN       NaN     6                 3.93
6   NaN       NaN     7                 6.67
7   NaN       NaN     9                 3.54
8   NaN       NaN    10                 9.29
